This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [ ]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'

from astropy.table import Table

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

import warnings
#We ignore warnings - this is a little dangerous but a huge number of warnings are generated by empty cells later
warnings.filterwarnings('ignore')

In [ ]:
SUFFIX = '20180301'
file_list = glob.glob('../../dmu32/dmu32_Herschel-Stripe-82/data/tiles/herschel-stripe-82_{}_cigale_*.fits'.format(SUFFIX))

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [ ]:
# good = {}
# for band in ['pacs_green', 'pacs_red', 'spire_250', 'spire_350', 'spire_500']:
#     good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) & 
#                                    ~master_catalogue['flag_{}'.format(band)])
#     good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
#                               master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [ ]:
# combined_good = np.sum(list(good.values()), axis=0) >= 2

In [ ]:
# print("Number of good sources: {}".format(np.sum(combined_good)))

# Main catalogue for CIGALE

In [ ]:
# best_catalogue = master_catalogue[combined_good].copy()

In [ ]:
# Correction for galactic extinction
# best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [ ]:
# Convertion to CIGALE format
# best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [ ]:
u_bands = [                                     "sdss_u"]
g_bands = ["megacam_g", "suprime_g", "decam_g", "sdss_g",                                   "gpc1_g"]
r_bands = ["megacam_r", "suprime_r", "decam_r", "sdss_r",                                   "gpc1_r"]
i_bands = ["megacam_i", "suprime_i", "decam_i", "sdss_i",                                   "gpc1_i"]
z_bands = ["megacam_z", "suprime_z", "decam_z", "sdss_z",                                   "gpc1_z"]
y_bands = ["megacam_y", "suprime_y", "decam_y",           "vista_y",  "ukidss_y",           "gpc1_y"] 
j_bands = [                                               "vista_j",  "ukidss_j", "wircam_j"]
h_bands = [                                               "vista_h",  "ukidss_h"]
k_bands = [                                               "vista_ks",           , "wircam_ks"]#"ukidss_k"]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [ ]:
# remove_unneeded_fluxes(g_bands)
# remove_unneeded_fluxes(u_bands)
# remove_unneeded_fluxes(r_bands)
# remove_unneeded_fluxes(i_bands)
# remove_unneeded_fluxes(z_bands)
# remove_unneeded_fluxes(y_bands)

In [ ]:
#best_catalogue.write("data_tmp/CDFS-SWIRE_cigale_best_extcor_20180129.fits")

# Catalogue using spectroscopic redshift

In [ ]:
for tile in file_list:
        
    file_num = tile.split('_')[4].split('.')[0]
    best_catalogue = Table.read(tile)



    best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

    print("Number of sources with z-spec in file {}: {}".format(file_num, len(best_catalogue)))

    # Correction for galactic extinction
    best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

    # Convertion to CIGALE format

    best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

    remove_unneeded_fluxes(g_bands)
    remove_unneeded_fluxes(u_bands)
    remove_unneeded_fluxes(r_bands)
    remove_unneeded_fluxes(i_bands)
    remove_unneeded_fluxes(z_bands)
    remove_unneeded_fluxes(y_bands)
    remove_unneeded_fluxes(j_bands)
    remove_unneeded_fluxes(h_bands)
    remove_unneeded_fluxes(k_bands)

    best_catalogue.write("data_tmp/tiles/herschel-stripe-82_cigale_all_extcor_zspec_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)